Import

In [ ]:
!pip install librosa pandas numpy scikit-learn joblib

In [6]:
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

Config

In [7]:
FRAME_SIZE=1024
HOP_LENGTH=512*2
SR = 22050
WD = 29696*2

Hàm lấy tên của các file trong dataset

In [8]:
def get_name_file_paths():
  name_audios = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
  path_names = []
  for name_audio in name_audios:
    for i in range(0,10):
      path_names.append(f"{name_audio}.{str(i).zfill(5)}.wav")
  return path_names

Hàm load file audio trong dataset

In [9]:
def load_audio(path_names):
  audio_files = []
  pre_path =  './dataset/'
  for path_name in path_names:
    audio_file, _ = librosa.load(f"{pre_path}{path_name}",sr=SR)
    audio_files.append(audio_file)
  return audio_files

Hàm chia nhỏ các audio thành nhiều cửa sổ 

In [10]:
def split_audio_into_frames(audio_file, frame_size, hop_length):
  frames = []
  for i in range(0, len(audio_file) - frame_size, hop_length):
    frame = audio_file[i:i + frame_size]
    frames.append(frame)
  return frames

Hàm trích rút đặc trưng

In [11]:
def extract_features(audio_file):
  sr = 22050
  chroma_stft = librosa.feature.chroma_stft(y=audio_file, sr=sr)

  chroma_stft_mean = np.mean(np.mean(chroma_stft, axis=1))
  chroma_stft_var = np.var(chroma_stft, axis=1)
  chroma_stft_mean_mean = np.mean(chroma_stft_mean)
  chroma_stft_var_mean = np.mean(chroma_stft_var)
  rms = librosa.feature.rms(y=audio_file).flatten()
  rms_mean = np.mean(rms)
  rms_var = np.var(rms)
  spectral_centroid = librosa.feature.spectral_centroid(y=audio_file, sr=sr).flatten()
  spectral_centroid_mean = np.mean(spectral_centroid)
  spectral_centroid_var = np.var(spectral_centroid)

  spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio_file, sr=sr).flatten()
  spectral_bandwidth_mean = np.mean(spectral_bandwidth)
  spectral_bandwidth_var = np.var(spectral_bandwidth)

  rolloff = librosa.feature.spectral_rolloff(y=audio_file, sr=sr).flatten()
  rolloff_mean = np.mean(rolloff)
  rolloff_var = np.var(rolloff)

  zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_file).flatten()
  zero_crossing_rate_mean = np.mean(zero_crossing_rate)
  zero_crossing_rate_var = np.var(zero_crossing_rate)

  harm, perc = librosa.effects.hpss(audio_file)
  harm_mean = np.mean(harm)
  harm_var = np.var(harm)
  perc_mean = np.mean(perc)
  perc_var = np.var(perc)
  mfccs = librosa.feature.mfcc(y=audio_file, sr=sr)
  mfccs_mean = np.mean(mfccs)
  mfccs_var = np.var(mfccs)
  data = []
  data.append(chroma_stft_mean_mean)
  data.append(chroma_stft_var_mean)
  data.append(rms_mean)
  data.append(rms_var)
  data.append(spectral_centroid_mean)
  data.append(spectral_centroid_var)
  data.append(spectral_bandwidth_mean)
  data.append(spectral_bandwidth_var)
  data.append(rolloff_mean)
  data.append(rolloff_var)
  data.append(zero_crossing_rate_mean)
  data.append(zero_crossing_rate_var)
  data.append(harm_mean)
  data.append(harm_var)
  data.append(perc_mean)
  data.append(perc_var)
  data.append(mfccs_mean)
  data.append(mfccs_var)
  return data


Hàm chia nhỏ thành các cửa sổ và trích rút trên từng cửa sổ

In [12]:
def split_extract_features(audio_file):
  data_fr = []
  frames = split_audio_into_frames(audio_file,frame_size=WD,hop_length=HOP_LENGTH*2)
  for frame in frames:
    data = extract_features(frame)
    data_fr.append(data)
  return data_fr

Hàm trích rút đặc trưng cho dataset

In [13]:
def extract_features_dataset():
    audio_names = get_name_file_paths()
    audio_files = load_audio(audio_names)
    data = []
    i=0
    for audio_file in audio_files:
        features = split_extract_features(audio_file)
        # Tạo DataFrame từ các đặc trưng và thêm cột định danh
        df = pd.DataFrame(features, columns=['chroma_stft_mean_mean','chroma_stft_var_mean','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harm_mean','harm_var','perc_mean','perc_var','mfccs_mean','mfccs_var'])
        df['name_file'] = audio_names[i]
        index = audio_names[i].find(".")
        name_type = audio_names[i][:index] if index != -1 else 'unknow'
        df['label'] = name_type
        i = i+1
        # Thêm DataFrame này vào danh sách dữ liệu
        data.append(df)
    final_df = pd.concat(data, ignore_index=True)
    final_df.to_csv('features.csv', index=False)

Hàm tính độ tương đồng

In [14]:
def euclidean_distance(point1, point2):
    distance = np.sqrt(np.sum((point1 - point2) ** 2))
    return distance
def euclidean_distance_arr(arr1,arr2,leg):
  sum = 0
  for i in range(leg):
   sum += euclidean_distance(arr1[i],arr2[i])
  return sum/leg
def compare_features(query_features, data_features):
    min_distance = float('inf')  # Khởi tạo khoảng cách nhỏ nhất với giá trị vô cực
    if(len(data_features) < len(query_features)) :
        temp = data_features
        data_features = query_features
        query_features = temp
    len_query = len(query_features)
    len_datafeatures= len(data_features)
    for i in range(len_datafeatures - len_query):
        # Lấy cụm cửa sổ đặc trưng tương ứng từ file trong dữ liệu
        data_window = data_features[i:i+len_query]
        # Tính khoảng cách Euclid giữa cụm cửa sổ đặc trưng của file trong dữ liệu và của file truy vấn
        distance = euclidean_distance_arr(np.array(data_window),np.array(query_features),len_query)
        # Cập nhật khoảng cách nhỏ nhất
        if distance < min_distance:
            min_distance = distance
    # Trả về khoảng cách nhỏ nhất
    similarity = 1 / (1 + min_distance)
    return similarity

Hàm tính toán truy vấn

In [15]:
def query_audio(data_feature,query_feature,top_k):
  name_files = get_name_file_paths()
  similarities = []
  for name_file in name_files:
    data_feature_cur= data_feature[data_feature['name_file']==name_file].drop(['name_file', 'label'], axis=1)
    similarity = compare_features(query_feature, data_feature_cur)
    similarities.append((name_file, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
  top_matches = similarities[:top_k]
  return top_matches

Hàm truy vấn và hiển thị 3 kết quả tương đồng nhất

In [16]:
from IPython.display import Audio
def display_result_query(audio_file,df_scaled,scaler):
 scaled_features_audio_query = scaler.transform(split_extract_features(audio_file))
 matching_files  = query_audio(df_scaled,scaled_features_audio_query,3)
 print("Top 3 audio most similar")
 for file_info in matching_files:
    file_path = './dataset/' + file_info[0]  
    confidence = file_info[1]  
    print(f"Phát tệp âm thanh {file_path}, Độ tin cậy: {confidence}")
    display(Audio(file_path, autoplay=True))  

Scale dataset

In [17]:

def handle_scale_dataset(df_features):
 scaler = MinMaxScaler()
 scaled_features = scaler.fit_transform(df_features)
 df_scaled_features = pd.DataFrame(scaled_features, columns=df_features.columns)
 return df_scaled_features,scaler
 

Tiền sử lý data, kết quả thu được file data đã được scale và scaler model

In [18]:
def prepossessing():
 extract_features_dataset()
 df = pd.read_csv("features.csv")
 df_metadata = df[['name_file', 'label']]
 df_features = df.drop(columns=['name_file', 'label'])
 df_scaled_features,scaler = handle_scale_dataset(df_features)
 df_scaled = pd.concat([df_metadata, df_scaled_features], axis=1)
 df_scaled.to_csv('scaled_data.csv', index=False)
 joblib.dump(scaler, 'scaler.pkl')
 

Giao diện

In [19]:
import ipyfilechooser as ipfc
from IPython.display import clear_output

def on_file_chosen(chooser,df_features_scaled,scaled):
    file_path = chooser.selected
    if file_path:
        clear_output(wait=True)
        display_widget_input(df_features_scaled,scaled)
        print("Audio query: ", file_path)
        display(Audio(file_path, rate=22050))
        audio_query, sr = librosa.load(file_path)
        display_result_query(audio_query,df_features_scaled,scaled)
# Kết nối sự kiện chọn file với hàm xử lý
def display_widget_input(df_features_scaled,scaled):
    chooser = ipfc.FileChooser()
    chooser.register_callback(callback=lambda chooser: on_file_chosen(chooser,df_features_scaled,scaled))
    print("Hãy chọn file audio cần tìm kiếm: ")
    display(chooser)


Chạy hàm tiền sửa lý để có file data scale và scaler model

In [20]:
# prepossessing()

Main

In [ ]:
def main():
 df_features_scaled = pd.read_csv("scaled_data.csv")
 loaded_scaler = joblib.load('scaler.pkl')
 display_widget_input(df_features_scaled,loaded_scaler)
main()

Hãy chọn file audio cần tìm kiếm: 


FileChooser(path='C:\Users\ACER\Desktop\dpt', filename='', title='', show_hidden=False, select_desc='Select', …

Audio query:  C:\Users\ACER\Desktop\dpt\audio_query\country.query30.wav


c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Top 3 audio most similar
Phát tệp âm thanh ./dataset/reggae.00001.wav, Độ tin cậy: 0.7404646477461782


Phát tệp âm thanh ./dataset/jazz.00001.wav, Độ tin cậy: 0.6859812939700344


Phát tệp âm thanh ./dataset/blues.00001.wav, Độ tin cậy: 0.6808691940199687
